In [1]:
import pandas as pd

In [2]:
data=pd.read_csv("./data/covid19_italy_region.csv")

In [3]:
data

,SNo,Date,Country,RegionCode,RegionName,Latitude,Longitude,HospitalizedPatients,IntensiveCarePatients,TotalHospitalizedPatients,HomeConfinement,CurrentPositiveCases,NewPositiveCases,Recovered,Deaths,TotalPositiveCases,TestsPerformed
0,0,2020-02-24 18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,0,0,0,0,0,5
1,1,2020-02-24 18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,0,0,0,0,0,0
2,2,2020-02-24 18:00:00,ITA,4,P.A. Bolzano,46.499335,11.356624,0,0,0,0,0,0,0,0,0,1
3,3,2020-02-24 18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,0,0,0,0,0,1
4,4,2020-02-24 18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,0,0,0,0,0,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352,352,2020-03-11 17:00:00,ITA,9,Toscana,43.769231,11.255889,87,54,141,173,314,54,5,1,320,2804
353,353,2020-03-11 17:00:00,ITA,4,P.A. Trento,46.068935,11.121231,28,4,32,42,74,24,3,0,77,527
354,354,2020-03-11 17:00:00,ITA,10,Umbria,43.106758,12.388247,7,5,12,32,44,7,2,0,46,340
355,355,2020-03-11 17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,2,0,2,17,19,2,0,1,20,99
